### Hybrid Search Langchain

In [1]:
!pip install --upgrade --quiet  pinecone 

In [1]:
api_key="pcsk_2odQAg_2Uaw8rJ5zvtrx1xSUi88TX9ECSKGZxyEgdgfWFAsWHGZu2auYy9PyAqPUy8h9gH"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model this 384 is using because we are going to use the hugging face embedding techniques which also convert into the vector embedding of 384 
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

/Users/dhruv/Desktop/GenAI Projects/Genai/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
index=pc.Index(index_name)
index

In [5]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
import warnings 
load_dotenv()
warnings.filterwarnings('ignore')

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/dhruv/Desktop/GenAI Projects/Genai/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/dhruv/Desktop/GenAI Projects/Genai/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/dhruv/Desktop/GenAI Projects/Genai/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/dhruv/Desktop/GenAI Projects/Genai/lib/python3.10/site-packages/traitlets/con

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [10]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 2995.93it/s]


In [11]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [12]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000020F55236530>, index=<pinecone.data.index.Index object at 0x0000020F5D2F4730>)

In [13]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

100%|██████████| 1/1 [00:04<00:00,  4.54s/it]


In [16]:
retriever.invoke("What city did i visit first")

[Document(page_content='In 2021, I visited New Orleans'),
 Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2023, I visited Paris')]